In [1]:
# necessary imports
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np

# for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [6]:
# define file paths
import os
if os.path.isdir('../Data'):
    train_path = '../Data/train/'
    val_path = '../Data/val/'
    test_path = '../Data/test/'
else:
    ! git clone https://github.com/hassan-ademola/Hamoye_capstone_project_smote.git
    train_path = 'Hamoye_capstone_project_smote/Data/train/'
    val_path = 'Hamoye_capstone_project_smote/Data/val/'
    test_path = 'Hamoye_capstone_project_smote/Data/test/'

In [7]:
# generate train and test sets from directories
train_ds = image_dataset_from_directory(train_path,label_mode='categorical',
                                        image_size=(256, 256))
val_ds = image_dataset_from_directory(val_path,label_mode='categorical',
                                      image_size=(256, 256))
test_ds = image_dataset_from_directory(test_path,label_mode='categorical',
                                       shuffle=False,image_size=(256, 256))

In [9]:
# load a sample image
from tensorflow.keras.preprocessing import image

img_path = train_path+'Buffalo/Buffalo_1.jpg'
img = image.load_img(img_path, target_size=(600, 600))
img

In [10]:
# make prediction with a pretrained model
from keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.applications.efficientnet import decode_predictions

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

base_model = EfficientNetB7(weights='imagenet')
pred = base_model.predict(x)
print('Predicted:', decode_predictions(pred))

In [11]:
from keras import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = EfficientNetB7(weights='imagenet', include_top=False,
                            input_shape=(256,256,3))

# freeze extraction layers
base_model.trainable = False

# add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(4, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# confirm unfrozen layers
for layer in model.layers:
    if layer.trainable==True:
        print(layer)

In [13]:
# create callback to use during training
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3,
                                            restore_best_weights=True)

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

# train the model on the new data for a few epochs
history = model.fit(train_ds,epochs=20,validation_data=val_ds,
                    callbacks=[early_stopping])

In [14]:
# save results
results = history.history

In [15]:
# plot results
import matplotlib.pyplot as plt
n_epochs = len(results['loss'])

plt.figure(figsize=[14,4])
plt.grid(True)
plt.subplot(1,2,1)
plt.plot(range(1, n_epochs+1), results['loss'], label='Training')
plt.plot(range(1, n_epochs+1), results['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.grid(True)
plt.subplot(1,2,2)
plt.plot(range(1, n_epochs+1), results['accuracy'], label='Training')
plt.plot(range(1, n_epochs+1), results['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [16]:
predictions = model.evaluate(test_ds)

In [17]:
# make predictions on the unseen data
predictions = model.predict(test_ds)
predictions

In [18]:
# save the index of the highest probability
predictions = predictions.argmax(axis=1)
predictions

In [19]:
# get the actual values
test_images = list(test_ds.unbatch().as_numpy_iterator())
y_true = np.array([i[1] for i in test_images])
y_true = y_true.argmax(axis=1)
y_true

In [20]:
# calculate f1_score
from sklearn.metrics import f1_score
f1_score(y_true,predictions,average='macro')

In [21]:
# get the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true,predictions)

The model misclassified a **buffalo** and **rhino** image. Let's see

In [25]:
import pandas as pd
comparison = pd.DataFrame({'actual':y_true,'predicted':predictions,
                           'same':y_true==predictions})
comparison.replace({0:'Buffalo',1:'Elephant',2:'Rhino',3:'Zebra'},inplace=True)
comparison[comparison.same==False]

In [23]:
image.array_to_img(test_images[250][0])

In [24]:
image.array_to_img(test_images[633][0])

In [ ]:
# uncomment the below to save the model
'''import pickle
pickle.dump(model, open('efficientnetb7.pkl', 'wb'))'''